In [1]:
# Preprocess + Validate: Chotot Motorbikes

import pandas as pd
import numpy as np
import re
from pathlib import Path
from datetime import datetime

DATA_DIR = Path('/Users/doananh/Documents/đồ án DS')
RAW_FILE = DATA_DIR / 'data_motobikes.xlsx - Sheet1.csv'
CLEAN_FILE = DATA_DIR / 'data_motobikes_clean.csv'
VALIDATION_FILE = DATA_DIR / 'validation_issues.csv'

# Load raw as strings for safe parsing

df_raw = pd.read_csv(
    RAW_FILE,
    dtype=str,
    on_bad_lines='skip',
    low_memory=False,
    encoding='utf-8'
)
print('Raw shape:', df_raw.shape)
df_raw.head(3)


Raw shape: (7208, 18)


,id,Tiêu đề,Giá,Khoảng giá min,Khoảng giá max,Địa chỉ,Mô tả chi tiết,Thương hiệu,Dòng xe,Năm đăng ký,Số Km đã đi,Tình trạng,Loại xe,Dung tích xe,Xuất xứ,Chính sách bảo hành,Trọng lượng,Href
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",66.000.000 đ,72.53 tr,85.14 tr,"Phường Bến Thành, Quận 1, Tp Hồ Chí Minh",Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,Piaggio,Vespa,2024,14000,Đã sử dụng,Tay ga,100 - 175 cc,Đang cập nhật,Bảo hành hãng,> 50 kg,https://xe.chotot.com/mua-ban-xe-may-quan-1-tp...
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,79.500.000 đ,62.76 tr,73.68 tr,"Phường Tân Định, Quận 1, Tp Hồ Chí Minh","_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...",Honda,SH,2019,28000,Đã sử dụng,Tay ga,100 - 175 cc,Đang cập nhật,Bảo hành hãng,> 50 kg,https://xe.chotot.com/mua-ban-xe-may-quan-1-tp...
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,37.000.000 đ,28 tr,32.86 tr,"Phường Cầu Kho, Quận 1, Tp Hồ Chí Minh",Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,Honda,Vision,2023,12000,Đã sử dụng,Tay ga,100 - 175 cc,Đang cập nhật,Bảo hành hãng,> 50 kg,https://xe.chotot.com/mua-ban-xe-may-quan-1-tp...


In [2]:
# Cleaning helpers

VI_NULLS = {None, np.nan, '', 'Đang cập nhật', 'dang cap nhat', 'đang cập nhật', 'NaN', 'nan'}
_price_re = re.compile(r'\d+[\d\.]*')
_tr_re = re.compile(r'([\d]+(?:[\.,][\d]+)?)\s*tr', re.IGNORECASE)
_num_re = re.compile(r'[\d]+(?:[\.,][\d]+)?')


def is_null_like(v):
    if v is None:
        return True
    s = str(v).strip()
    if s in VI_NULLS:
        return True
    if s.lower() in {'không rõ', 'unknown', 'n/a', 'none'}:
        return True
    return False


def normalize_text(v):
    if is_null_like(v):
        return None
    s = ' '.join(str(v).split())
    return s if s else None


def to_int(v):
    if is_null_like(v):
        return None
    s = re.sub(r'[^0-9-]', '', str(v))
    if s in {'', '-'}:
        return None
    try:
        return int(s)
    except Exception:
        return None


def parse_gia_vnd(v):
    if is_null_like(v):
        return None
    m = _price_re.search(str(v))
    if not m:
        return None
    digits = m.group(0).replace('.', '')
    try:
        return float(digits)
    except Exception:
        return None


def parse_tr(v):
    if is_null_like(v):
        return None
    s = str(v).replace(',', '.')
    m = _tr_re.search(s)
    if not m:
        return None
    try:
        return float(m.group(1))
    except Exception:
        return None


def parse_cc_numeric(s):
    """Parse engine capacity text to a numeric representative (cc).
    Examples: 'Dưới 50 cc' -> 50; '50 - 100 cc' -> 75; '100 - 175 cc' -> 137.5; 'Trên 175 cc' -> 200
    """
    if is_null_like(s):
        return None
    txt = str(s).lower()
    if 'dưới' in txt or '<' in txt:
        # Under 50 -> use 50 as proxy
        return 50.0
    if 'trên' in txt or '>' in txt:
        # Over 175 -> use 200 proxy
        return 200.0
    nums = [float(x.replace(',', '.')) for x in _num_re.findall(txt)]
    if len(nums) == 0:
        return None
    if len(nums) == 1:
        return nums[0]
    # Range -> midpoint
    return float((nums[0] + nums[1]) / 2.0)


def parse_weight_kg(s):
    """Parse weight like '> 50 kg' -> 50 (lower bound)."""
    if is_null_like(s):
        return None
    txt = str(s).lower()
    nums = [float(x.replace(',', '.')) for x in _num_re.findall(txt)]
    if not nums:
        return None
    # take the first number; if there's a '>' treat as lower bound
    return float(nums[0])


def parse_year(v):
    if is_null_like(v):
        return None
    s = str(v)
    if s.strip().startswith('trước'):
        return None
    return to_int(s)


def parse_address(addr):
    out = {"phuong": None, "quan": None, "tinh_thanh": None}
    s = normalize_text(addr)
    if not s:
        return out
    parts = [p.strip() for p in s.split(',')]
    if len(parts) >= 1:
        out['phuong'] = parts[0]
    if len(parts) >= 2:
        out['quan'] = parts[1]
    if len(parts) >= 3:
        out['tinh_thanh'] = parts[-1]
    return out


In [3]:
# Rename columns and normalize

col_map = {
    'id': 'id',
    'Tiêu đề': 'tieu_de',
    'Giá': 'gia_vnd_str',
    'Khoảng giá min': 'gia_min_tr_str',
    'Khoảng giá max': 'gia_max_tr_str',
    'Địa chỉ': 'dia_chi',
    'Mô tả chi tiết': 'mo_ta',
    'Thương hiệu': 'thuong_hieu',
    'Dòng xe': 'dong_xe',
    'Năm đăng ký': 'nam_dang_ky',
    'Số Km đã đi': 'so_km',
    'Tình trạng': 'tinh_trang',
    'Loại xe': 'loai_xe',
    'Dung tích xe': 'dung_tich',
    'Xuất xứ': 'xuat_xu',
    'Chính sách bảo hành': 'bao_hanh',
    'Trọng lượng': 'trong_luong',
    'Href': 'href',
}

df = df_raw[[c for c in df_raw.columns if c in col_map]].rename(columns=col_map).copy()

# Trim/normalize text
text_cols = ['tieu_de','dia_chi','mo_ta','thuong_hieu','dong_xe','tinh_trang','loai_xe','dung_tich','xuat_xu','bao_hanh','trong_luong','href']
for c in text_cols:
    if c in df.columns:
        df[c] = df[c].apply(normalize_text)

# Numeric conversions
if 'id' in df:
    df['id'] = df['id'].apply(to_int)

if 'gia_vnd_str' in df:
    df['gia_vnd'] = df['gia_vnd_str'].apply(parse_gia_vnd)

if 'gia_min_tr_str' in df:
    df['gia_min_tr'] = df['gia_min_tr_str'].apply(parse_tr)

if 'gia_max_tr_str' in df:
    df['gia_max_tr'] = df['gia_max_tr_str'].apply(parse_tr)

if 'nam_dang_ky' in df:
    df['nam_dang_ky'] = df['nam_dang_ky'].apply(parse_year)

if 'so_km' in df:
    df['so_km'] = df['so_km'].apply(to_int)

# Derive gia_vnd_final from range average when missing
if {'gia_vnd','gia_min_tr','gia_max_tr'}.issubset(df.columns):
    tr_to_vnd = lambda x: x * 1_000_000 if pd.notnull(x) else np.nan
    avg_tr = (df['gia_min_tr'] + df['gia_max_tr']) / 2.0
    df['gia_vnd_from_range'] = avg_tr.apply(tr_to_vnd)
    df['gia_vnd_final'] = df['gia_vnd']
    df.loc[df['gia_vnd_final'].isna(), 'gia_vnd_final'] = df['gia_vnd_from_range']

# Parse engine capacity and weight to numeric
if 'dung_tich' in df:
    df['dung_tich_cc'] = df['dung_tich'].apply(parse_cc_numeric)
if 'trong_luong' in df:
    df['trong_luong_kg'] = df['trong_luong'].apply(parse_weight_kg)

# Split address
for part in ['phuong','quan','tinh_thanh']:
    df[part] = None
if 'dia_chi' in df:
    addr = df['dia_chi'].apply(parse_address)
    for part in ['phuong','quan','tinh_thanh']:
        df[part] = addr.apply(lambda d: d.get(part))

# Feature: lengths
df['len_title'] = df['tieu_de'].fillna('').apply(len)
df['len_desc'] = df['mo_ta'].fillna('').apply(len)

print('Cleaned columns:', df.columns.tolist())
print('Shape:', df.shape)
df.head(5)


Cleaned columns: ['id', 'tieu_de', 'gia_vnd_str', 'gia_min_tr_str', 'gia_max_tr_str', 'dia_chi', 'mo_ta', 'thuong_hieu', 'dong_xe', 'nam_dang_ky', 'so_km', 'tinh_trang', 'loai_xe', 'dung_tich', 'xuat_xu', 'bao_hanh', 'trong_luong', 'href', 'gia_vnd', 'gia_min_tr', 'gia_max_tr', 'gia_vnd_from_range', 'gia_vnd_final', 'dung_tich_cc', 'trong_luong_kg', 'phuong', 'quan', 'tinh_thanh', 'len_title', 'len_desc']
Shape: (7208, 30)


,id,tieu_de,gia_vnd_str,gia_min_tr_str,gia_max_tr_str,dia_chi,mo_ta,thuong_hieu,dong_xe,nam_dang_ky,...,gia_max_tr,gia_vnd_from_range,gia_vnd_final,dung_tich_cc,trong_luong_kg,phuong,quan,tinh_thanh,len_title,len_desc
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",66.000.000 đ,72.53 tr,85.14 tr,"Phường Bến Thành, Quận 1, Tp Hồ Chí Minh",Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,Piaggio,Vespa,2024.0,...,85.14,78835000.0,66000000.0,137.5,50.0,Phường Bến Thành,Quận 1,Tp Hồ Chí Minh,50,181
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,79.500.000 đ,62.76 tr,73.68 tr,"Phường Tân Định, Quận 1, Tp Hồ Chí Minh","_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...",Honda,SH,2019.0,...,73.68,68220000.0,79500000.0,137.5,50.0,Phường Tân Định,Quận 1,Tp Hồ Chí Minh,39,284
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,37.000.000 đ,28 tr,32.86 tr,"Phường Cầu Kho, Quận 1, Tp Hồ Chí Minh",Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,Honda,Vision,2023.0,...,32.86,30430000.0,37000000.0,137.5,50.0,Phường Cầu Kho,Quận 1,Tp Hồ Chí Minh,47,278
3,4,Vespa Sprint 2019 -125- Đen Đỏ Sport -CHÍNH ...,45.000.000 đ,43.1 tr,50.6 tr,"Phường Bến Nghé, Quận 1, Tp Hồ Chí Minh",XE CÁ NHÂN BÁN - XE DO EM ĐỨNG TÊN CHÍNH C...,Piaggio,Vespa,2019.0,...,50.60,46850000.0,45000000.0,137.5,50.0,Phường Bến Nghé,Quận 1,Tp Hồ Chí Minh,50,318
4,5,Xe tay ga Yamaha Latte 125 – Đăng ký 2021,23.000.000 đ,17.02 tr,19.98 tr,"Phường Tân Định, Quận 1, Tp Hồ Chí Minh",🛵 Thông tin xe: Dòng xe tay ga cao cấp Yamaha ...,Yamaha,Latte,2021.0,...,19.98,18500000.0,23000000.0,137.5,50.0,Phường Tân Định,Quận 1,Tp Hồ Chí Minh,41,493


In [4]:
# Basic sanity constraints and validation log

issues = []
now_year = datetime.now().year

# 1) Price must be positive
if 'gia_vnd_final' in df:
    bad = df[(df['gia_vnd_final'].isna()) | (df['gia_vnd_final'] <= 0)]
    for i in bad.index[:10000]:  # cap to avoid too large log
        issues.append({'index': int(i), 'field': 'gia_vnd_final', 'value': df.at[i,'gia_vnd_final'], 'reason': 'missing_or_nonpositive'})

# 2) Kilometers in [0, 300k]
if 'so_km' in df:
    bad = df[(df['so_km'].notna()) & ((df['so_km'] < 0) | (df['so_km'] > 300_000))]
    for i in bad.index[:10000]:
        issues.append({'index': int(i), 'field': 'so_km', 'value': df.at[i,'so_km'], 'reason': 'out_of_range'})
    # mask extreme values
    df['so_km'] = df['so_km'].mask((df['so_km'] < 0) | (df['so_km'] > 300_000))

# 3) Year in [1980, now]
if 'nam_dang_ky' in df:
    bad = df[(df['nam_dang_ky'].notna()) & ((df['nam_dang_ky'] < 1980) | (df['nam_dang_ky'] > now_year))]
    for i in bad.index[:10000]:
        issues.append({'index': int(i), 'field': 'nam_dang_ky', 'value': df.at[i,'nam_dang_ky'], 'reason': 'out_of_range'})
    df['nam_dang_ky'] = df['nam_dang_ky'].mask((df['nam_dang_ky'] < 1980) | (df['nam_dang_ky'] > now_year))

# 4) Weight must be positive if present
if 'trong_luong_kg' in df:
    bad = df[(df['trong_luong_kg'].notna()) & (df['trong_luong_kg'] <= 0)]
    for i in bad.index[:10000]:
        issues.append({'index': int(i), 'field': 'trong_luong_kg', 'value': df.at[i,'trong_luong_kg'], 'reason': 'nonpositive'})

# 5) Minimal URL validity check
if 'href' in df:
    mask = df['href'].fillna('').str.startswith('http') == False
    bad = df[mask]
    for i in bad.index[:10000]:
        issues.append({'index': int(i), 'field': 'href', 'value': df.at[i,'href'], 'reason': 'invalid_url'})

val_df = pd.DataFrame(issues)
val_df.to_csv(VALIDATION_FILE, index=False)
print('Validation issues saved to:', VALIDATION_FILE, '; total:', len(val_df))


Validation issues saved to: /Users/doananh/Documents/đồ án DS/validation_issues.csv ; total: 324


In [5]:
# Deduplicate and save

before = len(df)
keys = [k for k in ['id','href','tieu_de'] if k in df.columns]
if keys:
    df = df.drop_duplicates(subset=keys, keep='first')
else:
    df = df.drop_duplicates(keep='first')
after = len(df)
print(f'Deduplicated {before-after} rows; new shape: {df.shape}')

# Save cleaned dataset
df.to_csv(CLEAN_FILE, index=False, encoding='utf-8')
print('Saved cleaned to:', CLEAN_FILE)


Deduplicated 0 rows; new shape: (7208, 30)
Saved cleaned to: /Users/doananh/Documents/đồ án DS/data_motobikes_clean.csv
